## 다음 뉴스

In [8]:
import requests
import lxml.html

url = 'https://search.daum.net/search?w=news&nil_search=btn&DA=NTB&enc=utf8&cluster=y&cluster_page=1&q=%EC%9D%B8%EA%B3%B5%EC%A7%80%EB%8A%A5'
res = requests.get(url)
res.status_code

200

In [9]:
root = lxml.html.fromstring(res.text)  # HTML 내용을 해석
for e in root.cssselect('span.f_nb a'):  # <span class="f_nb"><a>
    print(e.text_content())  # 찾은 영역의 텍스트
    print(e.attrib['href'])  # 링크된 주소

다음뉴스
http://v.media.daum.net/v/20191003200025938
다음뉴스
http://v.media.daum.net/v/20191003193209362
다음뉴스
http://v.media.daum.net/v/20191002093545888
다음뉴스
http://v.media.daum.net/v/20190930162818200
다음뉴스
http://v.media.daum.net/v/20190929093403160
다음뉴스
http://v.media.daum.net/v/20190929130511692


`.strip()`은 앞뒤의 공백을 제거

## 다음 영화

In [24]:
import re

url = 'https://movie.daum.net/moviedb/grade?movieId=133315&type=netizen&page={}'
data = []  # 리스트를 만든다
for p in range(1, 3):
    res = requests.get(url.format(p))
    root = lxml.html.fromstring(res.text)

    grade = root.cssselect('em.emph_grade')  # 별점
    review = root.cssselect('p.desc_review') # 영화평

    for g, r in zip(grade, review):  # 별점과 영화평을 짝을 짓는다
        star = g.text_content()
        # 정규표현식으로 눈에 안보이는 문자들을 빈 칸으로 변환
        comment = re.sub(r'\s+', ' ', r.text_content())
        data.append((star, comment))

import pandas
pandas.DataFrame(data).to_csv('카봇.csv')

## User-Agent

In [30]:
requests.get('https://onoffmix.com')

<Response [500]>

User-Agent: 직역하자면 사용자의 대리인, 즉 웹브라우저

크롬의 정보를 똑같이 전달해서, 크롬을 쓰고 있는 척한다

In [31]:
requests.get('https://onoffmix.com', 
             headers={
                 'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/77.0.3865.90 Safari/537.36'
             })

<Response [200]>

## 인코딩

In [46]:
res = requests.get('https://www.scourt.go.kr/scourt/index.html')

In [50]:
res.encoding = 'cp949'  # 인코딩 강제 지정 

In [51]:
root = lxml.html.fromstring(res.text)

In [52]:
for e in root.cssselect('title'):
    print(e.text_content())

대한민국 법원


## 네이버 카페

In [53]:
url = 'https://m.cafe.naver.com/ArticleList.nhn?search.clubid=19773565&search.menuid=15&search.boardtype=L'

In [54]:
res = requests.get(url)

In [56]:
root = lxml.html.fromstring(res.text)

실습: 게시물 주소 수집하기

In [64]:
import urllib.parse

In [65]:
for e in root.cssselect('a._articleListItem'):
    # 현재 주소 url과 상대 주소를 결합(join()
    print(urllib.parse.urljoin(url, e.attrib['href'])) 

https://m.cafe.naver.com/ArticleRead.nhn?clubid=19773565&articleid=89877&page=1&boardtype=L&menuid=15
https://m.cafe.naver.com/ArticleRead.nhn?clubid=19773565&articleid=89824&page=1&boardtype=L&menuid=15
https://m.cafe.naver.com/ArticleRead.nhn?clubid=19773565&articleid=89716&page=1&boardtype=L&menuid=15
https://m.cafe.naver.com/ArticleRead.nhn?clubid=19773565&articleid=89647&page=1&boardtype=L&menuid=15
https://m.cafe.naver.com/ArticleRead.nhn?clubid=19773565&articleid=89640&page=1&boardtype=L&menuid=15
https://m.cafe.naver.com/ArticleRead.nhn?clubid=19773565&articleid=89630&page=1&boardtype=L&menuid=15
https://m.cafe.naver.com/ArticleRead.nhn?clubid=19773565&articleid=89613&page=1&boardtype=L&menuid=15
https://m.cafe.naver.com/ArticleRead.nhn?clubid=19773565&articleid=89560&page=1&boardtype=L&menuid=15
https://m.cafe.naver.com/ArticleRead.nhn?clubid=19773565&articleid=89489&page=1&boardtype=L&menuid=15
https://m.cafe.naver.com/ArticleRead.nhn?clubid=19773565&articleid=89361&page=1&bo

In [66]:
article_url = 'https://m.cafe.naver.com/ArticleRead.nhn?clubid=19773565&articleid=89877&page=1&boardtype=L&menuid=15'

아무거나 네이버에서 검색한 후 그 주소를 referer로 넣어준다

In [71]:
res = requests.get(article_url, headers={
    'referer': 'https://search.naver.com/search.naver?sm=tab_hty.top&where=nexearch&query=%EA%B5%AD%EB%AF%BC%EB%8C%80%ED%95%99%EA%B5%90&oquery=%ED%96%89%EB%8B%98%EB%93%A4...+%EC%96%B4%EA%B7%B8%EB%A1%9C%EA%B0%80+%EC%95%84%EB%8B%88%EB%9D%BC+%EB%AC%B8%EC%83%81%EB%93%9C%EB%A6%BD%EB%8B%88%EB%8B%A4&tqi=UijVpsp0JXossaNEK7sssssssVw-466509'
})

In [73]:
root = lxml.html.fromstring(res.text)
for e in root.cssselect('#postContent'):
    print(e.text_content().strip())

좀더 원활한 답변을 위해 아래와 같은 형식으로 작성 하여주시길 부탁드리오며  게시의 셩격이 어긋난 광고, 비방성 글들은 삭제 될 수 있습니다. 본 게시판에 문의글을 작성하시기 위해서는 건강한 의자생활 캠페인 '스탠드업, 치얼업' 사이트에서 내용을 꼼꼼히 보시고 진심을 담은 캠페인 글을 남겨주셔야 합니다. https://socialmkt4.wixsite.com/sucu/forum/kaempein-eungweon-hanmadi캠페인은 의자를 사용하시는 분들께 의자에 대한 바른생각과 인식을 고취하는 목적으로 운영되고 있습니다. 캠페인 게시판에 응원글을 남겨주셨다면 아래에의 양식에 따라 글을 작성해 주세요.양식에 맞지 않는 글은 삭제되거나 이동됩니다.  1. 캠페인 사이트 응원글 작성 링크 : 2. 구입 예정일 : 1주일이내  3. 용도 및 사용시간 : 컴퓨터 관련 공부 8시간이내 4. 사용 연령 : 25 5. 구입희망 브랜드 및 모델 (국내, 해외 제품) : 사이즈오브체어, 시디즈 t50 알아봤습니다.6. 구입희망 가격대 :30~507. 신장 및 체중 : 183cm ,57kg8. 요 청 내 용 : 좋은 의자를 처음 사보려는데 제가 엄청 말라서 의자에 앉으면 몸을 휘는? 팔 받침에 턱을 자주 괴어요. 제 신장 때문에 불안해서 주말에 집 근처에 있는 시디즈 가서 앉아보려구 하는데 사이즈오브체어 반응이 더 좋더라구요... 그리고 집 책상이 줄자로 쟀을때 77정도 높은데 현재 의자는 너무 낮아서 빨리 바꾸려고요. 추천 부탁드립니다
